# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [1]:
from razdel import tokenize
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score,  f1_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML 

import nltk
from nltk.corpus import stopwords
from pprint import pprint

In [2]:
data = pd.read_csv('../Компьютерная лингвистика ВШЭ/labeled.csv')

In [3]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [4]:
data.toxic.value_counts(normalize=True)

0.0    0.66514
1.0    0.33486
Name: toxic, dtype: float64

In [5]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)

In [6]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [7]:
def razdel_token(text):
    tokens = list(tokenize(text))
    tokens = [token.text for token in tokens]
    return tokens

In [8]:
default_vectorizer = CountVectorizer()
X_1 = default_vectorizer.fit_transform(train.comment)
X_test_1 = default_vectorizer.transform(test.comment) 

In [9]:
custom_vectorizer = CountVectorizer(tokenizer = razdel_token)
X_2 = custom_vectorizer.fit_transform(train.comment)
X_test_2 = custom_vectorizer.transform(test.comment) 

In [10]:
y_train = train.toxic.values
y_test = test.toxic.values

In [11]:
clf_1 = LogisticRegression(C=0.1, class_weight='balanced')
clf_1.fit(X_1, y_train)

predictions_1 = clf_1.predict(X_test_1)

In [12]:
clf_2 = LogisticRegression(C=0.1, class_weight='balanced')
clf_2.fit(X_2, y_train)

predictions_2 = clf_2.predict(X_test_2)

c:\users\tanbe\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
print(classification_report(y_test, predictions_1, digits=4))
print(f1_score(y_test, predictions_1))

              precision    recall  f1-score   support

         0.0     0.9141    0.8418    0.8764       948
         1.0     0.7364    0.8482    0.7883       494

    accuracy                         0.8440      1442
   macro avg     0.8252    0.8450    0.8324      1442
weighted avg     0.8532    0.8440    0.8463      1442

0.7883349012229539


In [14]:
print(classification_report(y_test, predictions_2, digits=4))
print(f1_score(y_test, predictions_2))

              precision    recall  f1-score   support

         0.0     0.9124    0.8460    0.8779       948
         1.0     0.7407    0.8441    0.7890       494

    accuracy                         0.8454      1442
   macro avg     0.8265    0.8451    0.8335      1442
weighted avg     0.8536    0.8454    0.8475      1442

0.7890255439924314


__Кастомный классификатор работает немного лучше, чем дефолтный, судя по f-score, однако большой разницы между ними нет.__

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [15]:
data = np.array([
                [1, 1, 1, 0, 0, 0],
                [1, 1, 1, 0, 0, 0],
                [3, 0, 1, 1, 0, 0],
                [1, 0, 0, 1, 1, 0],
                [0, 0, 0, 0, 0, 1]])

In [16]:
N = data.shape[0]
result = []

for d in data:
    dig = []
    for word in range(len(d)):
        dig.append(d[word] / d.sum() * np.log10(N / np.count_nonzero(data.T[word])))
    result.append(dig)
    
result_table = pd.DataFrame(data=result, columns=['я', 'ты', 'и', 'только', 'не', 'он'], 
                           index=['я и ты', 'ты и я', 'я, я и только я', 'только не я', 'он'])
result_table

,я,ты,и,только,не,он
я и ты,0.032303,0.132647,0.07395,0.000000,0.00000,0.00000
ты и я,0.032303,0.132647,0.07395,0.000000,0.00000,0.00000
"я, я и только я",0.058146,0.000000,0.04437,0.079588,0.00000,0.00000
только не я,0.032303,0.000000,0.00000,0.132647,0.23299,0.00000
он,0.000000,0.000000,0.00000,0.000000,0.00000,0.69897


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

### CountVectorizer и Logistic Regression

In [17]:
count_vectorizer = CountVectorizer(tokenizer = razdel_token, stop_words = stopwords.words('russian'),
                                   max_df=0.05, min_df=2, max_features=12000, 
                                   ngram_range=(1, 2))
X = count_vectorizer.fit_transform(train.comment)
X_test = count_vectorizer.transform(test.comment)

In [18]:
lr = LogisticRegression(C=5, penalty='l2', solver='newton-cg')
lr.fit(X, y_train)

preds = lr.predict(X_test)

In [19]:
print(classification_report(y_test, preds, zero_division=0))
print(f1_score(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.86      0.90      0.88       948
         1.0       0.79      0.73      0.76       494

    accuracy                           0.84      1442
   macro avg       0.83      0.81      0.82      1442
weighted avg       0.84      0.84      0.84      1442

0.7581837381203801


In [20]:
probs = lr.predict_proba(X_test)

toxic_probs = [p[1] for p in probs]

comments = test.comment.tolist()

res = dict(zip(comments, toxic_probs))
res_sorted = {f'{k} : {v}': v for k, v in sorted(res.items(), key=lambda item: item[1])}

top_ten_lr = list(res_sorted)[-10:]

### Tfidf Vectorizer и Наивный байесовский классификатор

In [21]:
tfidf_vectorizer = TfidfVectorizer(tokenizer = razdel_token, stop_words = stopwords.words('russian'),
                                   max_df=0.05, min_df=2, max_features=12000, sublinear_tf=False, 
                                   ngram_range=(1, 2))

X = tfidf_vectorizer.fit_transform(train.comment)
X_test = tfidf_vectorizer.transform(test.comment)

In [22]:
mnb = MultinomialNB(fit_prior=False, alpha=1.1)
mnb.fit(X, y_train)

preds = mnb.predict(X_test)

In [23]:
print(classification_report(y_test, preds, zero_division=0))
print(f1_score(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.87      0.90      0.89       948
         1.0       0.80      0.75      0.77       494

    accuracy                           0.85      1442
   macro avg       0.84      0.83      0.83      1442
weighted avg       0.85      0.85      0.85      1442

0.7740585774058577


In [24]:
probs = mnb.predict_proba(X_test)

toxic_probs = [p[1] for p in probs]

comments = test.comment.tolist()

res = dict(zip(comments, toxic_probs))
res_sorted = {f'{k} : {v}': v for k, v in sorted(res.items(), key=lambda item: item[1])}

top_ten_mnb = list(res_sorted)[-10:]

In [25]:
top_ten_lr

['Да нахуй ты мне это скинул? Если я теку от неё, естественно я пересмотрел её видосы. Но самый кайф на канале молодого человека, где он её за жопу хватает и пальцы в рот суёт. Если меня привлекают такие тян, я быдло?\n : 0.9999972553045907',
 'Ну а что такого то? Ясен хуй если он завел трактор то особой любви к рашке он не испытывал. Высказывает свое мнение. Все правильно говорит. Разбил розовые очки в один прекрасный день и увидел все как есть. Что он не так написал? Что люди тут друг друга не ненавидят? Все правильно пишет. Целиком читать не буду, мне времени жалко.\n : 0.9999990052746215',
 'Двач. Ну это пиздец, конечно. Раньше я его уважал, но теперь пошел он в пизду. На дваче такой зашквар сидеть и что самое смешное многие популярные конченные блогеры сидят на дваче, но я уж надеялся, что хотя бы Гуфен не такой, но раз он сидит на этой параше, то похуй.\n : 0.9999994332276646',
 'Плюсы: -Какие же хохлы дегенераты, пиздец просто. Минусы: -Какие же хохлв дегенераты, пиздец просто.\

In [26]:
top_ten_mnb

['Плюсы: -Какие же хохлы дегенераты, пиздец просто. Минусы: -Какие же хохлв дегенераты, пиздец просто.\n : 0.9898849439540488',
 'Какие же пиндосы дегенераты, пиздец просто.\n : 0.9899341573053712',
 'русня не умеет в готовку ... ведь поджарить хохла может только хохол\n : 0.9902951521677654',
 'Блять если тупой, то не иди в Бауманку... Пошел бы в ПТУ тогда уж.\n : 0.9906903903554602',
 'Какие же хохлы всадники, пиздец просто.\n : 0.991642883238058',
 'Какие же хохлы хайзенберги, пиздец просто.\n : 0.991642883238058',
 'Дебил блядь, стреляться в висок надо, полудурок ебанутый\n : 0.9922842691340125',
 'соси как хохлы сосут хохол\n : 0.9932969822313628',
 'Но Трамп и в правда дегенерат. Жаль, что этот клоун победил и мы теперь живем в пиздеце. Блять, если бы Клинтон сейчас была у власти, то в мире не было бы такого трэша. В рот ебал промыток с альт-райтом головного мозга, вы же нихуя своей головой думать не можете, животные, мемов про сжв насмотрелись теперь строите из себя дохуя консер

__У меня совпало три комментария в двух разных классификаторах. Я бы сказала, что CountVectorizer и Logistic Regression выбрал более токсичные комментарии. У Баесовского классификатора в топ токсичных комментариев попали такие, где есть слово "хохлы".__